In [ ]:
import pandas
import re
import nltk
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from gensim.models import KeyedVectors

# nltk.download('punkt_tab')

nome_file = "./DATI/Dataset.csv"
dati = pandas.read_csv(nome_file, encoding="utf-8", sep=",")
dati.columns

In [150]:
def preprocessa_testo (testo) -> str:

    # RIMUOVERE I LINK E GLI AT
    testo = re.sub(r"http\S+", "", testo) 
    testo = re.sub("@\S+", "", testo)
    testo = re.sub("[^A-Za-z\à\è\é\ì\ò\ù]+", " ", testo)
    
    parole_token = nltk.word_tokenize(testo)
    parole_token = [n.lower().strip() for n in parole_token]
    parole_token = [w for w in parole_token if not w in stopwords.words("italian")]

    return parole_token

dati["testi_puliti"] = dati.text.apply(lambda x: preprocessa_testo(x))

frasi = list(dati["testi_puliti"])

In [ ]:
from gensim.models import Word2Vec

# Inizializzaimo il Word2Vec
# https://radimrehurek.com/gensim/models/word2vec.html
modello = Word2Vec(vector_size=100,  # Dimensione degli embedding (100-200 per dataset piccoli/medi)
                   window=5,         # Contesto di 5 parole (ideale per commenti brevi)
                   min_count=5,      # Ignora parole con <5 occorrenze (riduce rumore)
                   sg=0,             # Modalità CBOW (più veloce e funziona bene con poco testo)
                   hs=0,             # Disabilita Hierarchical Softmax (usa Negative Sampling)
                   negative=10,      # Campiona 10 parole negative (bilancio qualità-velocità)
                   epochs=30,        # Più epoche per dataset piccoli (default=5)
                   workers=4,        # Threads per training parallelo
                   alpha=0.025,      # Learning rate iniziale
                   min_alpha=0.0001, # Learning rate finale (decresce linearmente)
                   cbow_mean=1       # Media dei vettori di contesto (default)
                   )

# Costruiamo il vocabolario
modello.build_vocab(frasi)

# Addestriamo il modello
modello.train(frasi,
              total_examples=modello.corpus_count,
              epochs=1000)

# Salviamo e ricarichiamo il nuovo modello + costruzione vocabolario
modello.save("nuovoword2vec01.model")

In [3]:
nuovo_modello = Word2Vec.load("nuovoword2vec01.model")
parole = list(nuovo_modello.wv.index_to_key)

In [ ]:
# Interrogazione del modello
# vettore = nuovo_modello.wv["parola"]
# print(vettore)
correlati = nuovo_modello.wv.most_similar("uomo", topn = 25)
print(*correlati, sep="\n")

In [ ]:
word_vectors = nuovo_modello.wv  # wv contiene i KeyedVectors
similarity = word_vectors.similarity('maschio', 'criminale')
print(similarity)

In [9]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib qt

# Carica il mio modello
model = Word2Vec.load("nuovoword2vec01.model")  
# Lista keyword
parole = ['maschio', 'maschile', 'femminile', 'patriarcato',
          'violenza', 'criminale', 'femminicidio',  'italiani', 'immigrato', 'destra', 'sinistra']

translation = ['Male', 'Masculine', 'Feminine', 'Patriarchy',
          'Violence', 'Criminal', 'Femicide',  'Italians', 'Immigrant', 'Right-wing', 'Left-wing'] 

# Crea un array vuoto NxN
matrice_similarita = np.zeros(
    (len(parole), len(parole))
)
# Calcolo matrice di similarità effettiva
# Uso enumerate per avere simultaneamente sia gli indici sia le parole
for i, word1 in enumerate(parole):
    for j, word2 in enumerate(parole):
        matrice_similarita[i][j] = model.wv.similarity(word1, word2)
        if matrice_similarita[i][j] > 0.99:
            matrice_similarita[i][j] = 0
          
# Stabilisco una soglia per visualizzare i valori in matrice
soglia = 0.11
# Crea una maschera booleana per i valori sotto soglia
mask_sotto_soglia = matrice_similarita <= soglia

# Preparazione delle annotazioni sulla griglia
# np.where trova gli indici degli elementi maggiori di soglia
# Se un valore è maggiore di soglia, viene arrotondato a due decimali
# Se un valore è minore o uguale a soglia, viene sostituito con ""
annotations = np.where(
    matrice_similarita > soglia,
    np.round(matrice_similarita, 2),  # Mostra 2 decimali
    ""                                # Stringa per valori sotto soglia
)

# Crea una nuova figura per un grafico con una dimensione specifica in pollici
plt.figure(figsize=(12, 10))

# Importiamo lo stile Seaborn
# font_scale=1.0, mantiene la dimensione standard.
# font_scale=1.5, aumenta i font del 50%
sns.set(font_scale=0.9)

# Creazione heatmap come oggetto Axes di Matplotlib
# Il nome ax è convenzionale
ax = sns.heatmap(
    matrice_similarita,
    mask=mask_sotto_soglia,  # col parametro mask nascondiamo alcune celle
                             # che abbiamo memorizzato nella maschera booleana
    annot=annotations,       # annotazioni sono i valori della matrice filtrati
    fmt="",                  # annotazioni come sono fornite in annot,
                             # senza formattazione.
    cmap="Reds",             # modello colore
    # vmin=soglia,
    # vmax=1,
    xticklabels=translation,
    yticklabels=translation,
    linewidths=1,
    cbar_kws={'label': 'Word2Vec Similarity'} # Etichetta colorbar
)

# Personalizzazione grafico
ax.xaxis.tick_top()
ax.xaxis.set_label_position('top')

plt.tight_layout() # Regola automaticamente gli spazi tra gli elementi del grafico
plt.show()         # Mostra il grafico

In [10]:
# RIDURRE I DATI DI UN MODELLO W2V

# Importiamo le istruzioni per la riduzione dei dati: t-SNE 
from sklearn.manifold import TSNE
# Importiamo una libreria per la gestione degli array
import numpy as np

# Creiamo una funzione che riduce le dimensioni del modello
def riduci_dimensioni(nuovo_modello):
    
    numero_dimensioni = 2

    vettori = []        # Lista vuota di vettori
    etichette = []      # Lista vuota di parole
    
    for parola in nuovo_modello.wv.index_to_key:
        vettori.append(nuovo_modello.wv[parola])
        etichette.append(parola)

    # Un array è una lista che può contenere solo un tipo di elementi 
    # Gli array possono essere scritti in memoria in modo efficiente
    # .asarray trasforma le liste in array
    vettori = np.asarray(vettori)
    etichette = np.asarray(etichette)

    # t-distributed Stochastic Neighbor Embedding (t-SNE)
    # random_state corrisponde al seed (None = random)
    # .fit_transform genera le nuove coordinate dei fattori
    modello_tsne = TSNE(n_components=numero_dimensioni, random_state=2)
    vettori = modello_tsne.fit_transform(vettori)

    # Vettori e una lista di liste da due valori
    # Ritagliamo per x il 1° e per y il 2° valore 
    x_vals = [v[0] for v in vettori]
    y_vals = [v[1] for v in vettori]

    return x_vals, y_vals, etichette

x_vals, y_vals, etichette = riduci_dimensioni(nuovo_modello)

In [22]:
# RAPPRESENTAZIONE GRAFICA DI UN EMBEDDED MODEL

def disegna_modello(x_vals, y_vals, etichette):
    import matplotlib.pyplot as plt
    import random

    # 'serif' (es. Times New Roman, Georgia)
    # 'sans-serif' (es. Arial, Helvetica, Verdana)
    # 'monospace' (es. Courier New, Consolas)
    # 'cursive' (es. Comic Sans MS)
    plt.rcParams['font.family'] = 'serif'  # Imposta il font globale
    plt.rcParams['font.serif'] = ['Georgia']
    
    random.seed(12345)
    global AVx
    global AVy
    AVx = []
    AVy = []

    # Inizializza il grafico
    plt.figure(figsize=(10,10))
    %matplotlib qt
    
    # Sceglie il tipo di grafico
    plt.scatter(x_vals, y_vals, s=10, alpha=0.3)

    for etica in range(len(etichette)):
        if abs(x_vals[etica]) > 2 or abs(y_vals[etica]) > 2:
        # I file Excel degli assi si riempiono se metti soglie basse
            plt.annotate(etichette[etica], 
            (x_vals[etica], y_vals[etica]),
            color="black", fontsize=13)

            duox = [x_vals[etica], etichette[etica]]
            duoy = [y_vals[etica], etichette[etica]]

            AVx.append(duox)
            AVy.append(duoy)

    # parole_da_cercare
    parole_chiave = ["uomo", "maschio", "donna", "criminale", "italiani", "femminile",
                     "patriarcato", "destra", "sinistra", "stalker", "vittima",
                     "immigrati", "violenza", "ragazzo", "giornalismo", "diritti",
                     "maschile", "femminicidio", "ideologia", "politica", "stranieri", "giulia"]

    parole_chiave_sub = ["man", "male", "woman", "criminal", "italians", "feminine",
                     "patriarchy", "right-wing", "left-wing", "stalker", "victim",
                     "immigrants", "violence", "boy", "journalism", "rights",
                     "masculine", "femicide", "ideology", "politics", "foreigners", "giulia"]

    offsety = 0
    
    for i in range(len(parole_chiave)):
        indiceextra = np.where(etichette==parole_chiave[i])
        indiceextra = indiceextra[0][0]
        if parole_chiave[i]=="violenza":
            offsety = 0.4
        elif parole_chiave[i]=="maschile":
            offsety = 0.2
        elif parole_chiave[i]=="immigrati":
            offsety = 0.2
        plt.annotate(parole_chiave_sub[i], # etichette[indiceextra]
                     (x_vals[indiceextra], y_vals[indiceextra]+offsety),
                     color="black", fontsize=25, fontweight='bold'
                    )
        offsety = 0

    plt.axhline(0, color='black', linewidth=1) # origine x, colore, spessore
    plt.axvline(0, color='black', linewidth=1) # origine y, colore, spessore
    plt.tick_params(axis='both', which='major', labelsize=20) 
    
    plt.savefig("Grafico.png", dpi=3840)

disegna_modello(x_vals, y_vals, etichette)

Asse_X = pandas.DataFrame(sorted(AVx))
Asse_X.to_excel("AsseX.xlsx")
Asse_Y = pandas.DataFrame(sorted(AVy))
Asse_Y.to_excel("AsseY.xlsx")